# IAM Role

In this notebook, we review the concept of IAM role and set up one to be used for later notebooks in `/sagemaker-fundamentals`. An IAM role is an identity associated with your AWS account that has pre-configured permission policies that determine what this role can do and with respect to your AWS resources. For example, you can define a role that can do *everything* to your S3 resource but nothing else. Such an role is useful if it is *assumed* by a program that synchronize your data from your local machine to an S3 bucket. You can be certain that this program would not accidently create EC2 instances that incur higher costs. 

One application of IAM role is: it grants permissions to AWS services (e.g. SageMaker) to procure resources on your behalf. When you use an AWS service(e.g. SageMaker), you can define a role that the service can assume on your behalf to access the AWS resources. For example, SageMaker service needs to access S3 buckets, EC2 instances, Elastic Container Registries etc. To avoid incurring too much compute cost, you can define a role that is able to create only low cost EC2 instances. When SageMaker assumes this role and runs your ML workflow, you can estimate the upper bound of the compute cost based on the EC2 policy of the role. 

For more extensive readings on IAM role, refer to the [AWS documentation](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_terms-and-concepts.html#iam-term-role-chaining)

## Environments to run this notebook

1. If you are running this notebook from an EC2 instance, then you need to make sure `AWS_PROFILE` environment variable is set to `default`. 

2. If you are running this notebook on your local machine, you will need to install and configure aws command line interface. Follow [this link](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) to do so.

The IAM user/role running this notebook should at least have `iam:CreateRole` and `iam:DeleteRole` and `sts:AssumeRole` permission. 

I do not recommend to run it on an SageMaker Notebook Instance or Studio, because they automatically created an IAM role for you and the whole point of this notebook is to create one manually from scratch. 

Moreover, we will use `boto3` to interface with AWS API, make sure you have it installed. 

## Create an IAM Role

When you create an IAM role you need to specify
1. Which AWS entities (users or services) you trust to assume this role
2. What permissions this role has

1 is determined by a *trust policy* and 2 is determined by a *permission* policy. 

The entity that you entrust to assume the role are refered as *Principal* 

In [ ]:
import boto3  # your python gate way to all aws services
import pprint # print readable dictionary
import json
import time

pp = pprint.PrettyPrinter(indent=1)
iam = boto3.client('iam')

In [ ]:
# get the ARN of the user
caller_arn = boto3.client('sts').get_caller_identity()['Arn']

def create_execution_role(role_name="basic-role"):
    """Create an service role to procure services on your behalf
    
    Args:
        role_name (str): name of the role
    
    Return:
        dict
    """    
    # if the role already exists, delete it
    
    # Note: you need to make sure the role is not
    # used in production, because the code below
    # will delete the role and create a new one
    role = None
    for rol in iam.list_roles()['Roles']:
        if rol['RoleName'] == role_name:
            # detach policy from the role before deleting it
            role = boto3.resource('iam').Role(role_name)
            
            for p in role.attached_policies.all():
                role.detach_policy(PolicyArn=p.arn)
            break
    
    # Trust policy document
    trust_relation_policy_doc = {
      "Version": "2012-10-17",
      "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "AWS": caller_arn, # Allow caller to take this role
            "Service": [
              "sagemaker.amazonaws.com" # Allow SageMaker to take the role
            ],
          },
          "Action": "sts:AssumeRole",
        }
      ]
    }
    
    
    if role is not None:
        iam.delete_role(RoleName=role.name)
    
    res = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_relation_policy_doc)
    )
    return res

The trust policy above says we entrust the user of current boto3 session (most likely yourself) and SageMaker to assume this role. 

In [ ]:
role_res = create_execution_role()
pp.pprint(role_res)

Now, let's give the role some permissions. The dictionary below is an example of policy permission. It says: allow the role to list buckets under the AWS account.

In [ ]:
basic_s3_permission = {
    "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:List*" # Allow the role to perform list related actions, i.e read access
                    #"s3:*" 
                ],
                "Resource": [
                    "arn:aws:s3:::*" 
                ]
            }
        ]
    }

In [ ]:
def attach_permission(role_name, policy_name, policy_doc):
    """Attach a basic permission policy to the role"""

    # Create the policy
    # If the policy with policy name $policy_name already exists,
    # then we need to delete it first
    
    # Note: you need to make sure that you do not have a policy 
    # with $policy_name in production, because we will delete it
    # and create a new one with the policy document given by 
    # $policy_doc
    
    policy = None
    for p in iam.list_policies()['Policies']:
        if p['PolicyName']==policy_name:
            # Before we delete the policy, we need to detach it
            # from all IAM entities 
            policy = boto3.resource('iam').Policy(p['Arn'])
            
            # 1. detach from all groups
            for grp in policy.attached_groups.all():
                policy.detach_group(GroupName=grp.name)
                
            # 2. detach from all users
            for usr in policy.attached_users.all():
                policy.detach_user(UserName=usr.name)
            
            # 3. detach from all roles
            for rol in policy.attached_roles.all():
                policy.detach_role(RoleName=rol.name)
                
            break
    
    if policy is not None:
        iam.delete_policy(PolicyArn=policy.arn)   
    
    # create a new policy
    policy = iam.create_policy(
        PolicyName=policy_name,
        PolicyDocument=json.dumps(policy_doc))['Policy']
    
    # attach the policy to the role
    res = iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn=policy['Arn']
        )
    return res

In [ ]:
perm_res = attach_permission(
    role_name=role_res['Role']['RoleName'],
    policy_name='BasicS3Policy',
    policy_doc=basic_s3_permission
    )

pp.pprint(perm_res)

# Allow 15 seconds for the update to propagate
time.sleep(15)

## Test your role

Now, you can verify that the role we just created (`basic-role`) is allowed to list all S3 buckets under your account and it is not allowed to do anything else with your AWS resource. 

In [ ]:
# Create a boto3 session with credentials of basic-role
import time

now = str(time.time()).split('.')[0]

obj = boto3.client('sts').assume_role(
    RoleArn=role_res['Role']['Arn'],
    RoleSessionName=now
)

cred=obj['Credentials']

sess = boto3.session.Session(
    aws_access_key_id=cred['AccessKeyId'],
    aws_secret_access_key=cred['SecretAccessKey'],
    aws_session_token=cred['SessionToken']
    )

# initiate an S3 client from the session
s3 = sess.client('s3')

# list buckets 
pp.pprint(s3.list_buckets())

In [ ]:
# Try to create a bucket with the S3 client
# It is expected to fail, because basic-role 
# has no permission to create bucket 

import time

def create_bucket(s3_client):
    try:
        now = str(time.time()).split(".")[0]
        res = s3_client.create_bucket(
            Bucket='bucket-{}'.format(now), # bucket name, needs to be globally unique 
            CreateBucketConfiguration={
                "LocationConstraint": sess.region_name
            }
        )
        
        return res
    except Exception as e:
        print(e)
        return e

# expect to fail
create_bucket(s3)

Now, let's promote the role `basic-role` to allow it to create bucket. 

In [ ]:
create_bucket_permission = {
    "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:CreateBucket" 
                ],
                "Resource": [
                    "arn:aws:s3:::*" 
                ]
            }
        ]
    }

Note: attached policies are incremental, this means if we attach `create_bucket_permission` to `basic-role`, the effect of `basic_s3_permission` is still in place. So `basic-role` is still able to perform `List*` actions. 

In [ ]:
perm_res = attach_permission(
    role_name=role_res['Role']['RoleName'],
    policy_name='CreateBucket',
    policy_doc=create_bucket_permission
    )

pp.pprint(perm_res)
time.sleep(15)

In [ ]:
# You don't even need to create a new session
res = create_bucket(s3)
location = res['Location']
bucket_name = location.split('//')[1].split('.')[0]

Now, let's delete the bucket we just created. First, you will need to grant `basic-role` the permission to delete bucket. 

In [ ]:
delete_bucket_permission = {
    "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:DeleteBucket" 
                ],
                "Resource": [
                    "arn:aws:s3:::*" 
                ]
            }
        ]
    }

perm_res = attach_permission(
    role_name=role_res['Role']['RoleName'],
    policy_name='DeleteBucket',
    policy_doc=delete_bucket_permission
    )

pp.pprint(perm_res)
time.sleep(15)

In [ ]:
res = s3.delete_bucket(Bucket=bucket_name)
pp.pprint(res)

## Amazon Managed Policies

So far you have learnt how to create an IAM role and how to grant it permissions on your AWS resources via IAM polices. The example you went through above uses S3 buckets as an example service, but the same idea can be generalized when you need to define more complicated policies on more services. 

You do not always need to define your own policies. Amazon maintains a list of commonly used policies. You can view them through the [console](https://console.aws.amazon.com/iam/home?region=us-west-2#/policies) or you within this notebook by calling `ListPolicies` API of IAM service. 

In [ ]:
# List all S3 related policies
for p in iam.list_policies(Scope='AWS')['Policies']:
    if 'S3' in p['PolicyName']:
        pp.pprint(p)
        print("="*80)

One S3 policy maintained by Amazon is called `AmazonS3FullAccess`. As you expect, this policy grants the role to perform all actions on your S3 resources. Analogously, `AmazonEC2FullAccess` policy grants the role to perform all actions on your EC2 resources and `AmazonSageMakerFullAccess`
provides full access to Amazon SageMaker via the AWS Management Console and SDK. Also provides select access to related services

Policies are versioned. This gives you the abilitity to iteratively adjust the permissions to an execution role without changing the name of the policy. 

In [ ]:
# View AmazonSageMakerFullAccess policy
sagemaker_full = iam.get_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess')

In [ ]:
versions = iam.list_policy_versions(
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess')
pp.pprint(versions['Versions'][0])

The latest version for `AmazonSageMakerFullAccess` policy is `v19`. 

In [ ]:
# view the latest version of AmazonSageMakerFullAccess policy
pp.pprint(
    iam.get_policy_version(
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
        VersionId='v19'
    )
)

For most use cases of Amazon SageMaker, the `AmazonSageMakerFullAccess` policy would be sufficient. 

In [ ]:
# Attach AmazonSageMakerFullAccess to basic-role

res = iam.attach_role_policy(
    RoleName=role_res['Role']['RoleName'],
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
)

pp.pprint(res)

## Clean up

If you only plan to use the role to run certain application once, then it is a good practice to delete the role. If you plan to delete a role, make sure **you have detached all policies associated with it** and you **do not** have any Amazon EC2 instances running with the role you are about to delete. Deleting a role or instance profile that is associated with a running instance will break any applications running on the instance.



In [ ]:
# detach attached policies
attached_policies = iam.list_attached_role_policies(RoleName='basic-role')['AttachedPolicies']
for p in attached_policies:
    iam.detach_role_policy(
        RoleName=role_res['Role']['RoleName'],
        PolicyArn=p['PolicyArn']
    )

# delete the role
res = iam.delete_role(
   RoleName=role_res['Role']['RoleName']
)

pp.pprint(res)